In [ ]:
import keras
import pandas
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
df = pandas.read_csv("./sales_and_features.csv").drop(columns=["Date"])
# df = pandas.read_csv("data/pollution.csv", header=0, index_col=0)

In [ ]:
y = df.Sales_size
X = df.iloc[:,1:]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = pandas.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
df_X = format_data(scaled, 1, 1)
X = df_X.values.reshape(df_X.shape[0], 1, df_X.shape[1])
y = y[:-2]

In [ ]:
from matplotlib import pyplot
values = df.values
groups = list(range(1,11))
i = 1
pyplot.figure(figsize=(10,10))
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(values[:, group])
    pyplot.title(df.columns[group], y=0.5, loc='right')
    i += 1
pyplot.show()

In [ ]:
def format_data(df, n_in ,n_out):
    cols, names = list(), list()
    n_vars = df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pandas.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

In [ ]:
def create_train_test(df):
    values = df.values
    n_train_hours = 365 * 24
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]
    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    reutrn (train_X, train_y, test_X, test_y)

In [ ]:
df.columns

In [ ]:
df = pandas.read_csv("data/pollution.csv", header=0, index_col=0)
values = df.values
encoder = LabelEncoder()
values[:,8] = encoder.fit_transform(values[:,8])
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

dfs = dict()
for param in range(1,10):
    df = format_data(pandas.DataFrame(scaled), param, 1)
    train_X, train_y, test_X, test_y = create_train_test(df)
    

In [ ]:
def fit_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=0, shuffle=False)
    return history

In [ ]:
history.history['val_loss']

In [ ]:
def show_history(history):
    %matplotlib inline
    # from matplotlib import pyplot
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()

In [192]:
from sklearn import metrics
from sklearn.metrics import make_scorer

features = X.shape[2]
def create_model(units=10):
    model = Sequential()
    model.add(LSTM(10, input_shape=(1,features)))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
    return model
param_grid = {"units":[5,10]}
model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=50, verbose=1, validation_split=0.25)
# gridSearchCV = GridSearchCV(estimator=model, param_grid = param_grid, scoring=make_scorer(metrics.mean_absolute_error),  n_jobs=-1, cv=2, verbose=0)
gridSearchCV = GridSearchCV(estimator=model, param_grid = param_grid, scoring='neg_mean_absolute_error',  n_jobs=-1, cv=2, verbose=0)
grid_result = gridSearchCV.fit(X, y)

Train on 820 samples, validate on 274 samples
Epoch 1/50
820/820 [==============================] - 2s 2ms/step - loss: 6.5638 - mean_absolute_error: 6.5638 - val_loss: 7.2020 - val_mean_absolute_error: 7.2020
Epoch 2/50
820/820 [==============================] - 0s 49us/step - loss: 6.4722 - mean_absolute_error: 6.4722 - val_loss: 7.0879 - val_mean_absolute_error: 7.0879
Epoch 3/50
820/820 [==============================] - 0s 60us/step - loss: 6.3647 - mean_absolute_error: 6.3647 - val_loss: 6.9476 - val_mean_absolute_error: 6.9476
Epoch 4/50
820/820 [==============================] - 0s 48us/step - loss: 6.2315 - mean_absolute_error: 6.2315 - val_loss: 6.7700 - val_mean_absolute_error: 6.7700
Epoch 5/50
820/820 [==============================] - 0s 50us/step - loss: 6.0644 - mean_absolute_error: 6.0644 - val_loss: 6.5473 - val_mean_absolute_error: 6.5473
Epoch 6/50
820/820 [==============================] - 0s 48us/step - loss: 5.8599 - mean_absolute_error: 5.8599 - val_loss: 6.2782

820/820 [==============================] - 0s 63us/step - loss: 1.0369 - mean_absolute_error: 1.0369 - val_loss: 0.6780 - val_mean_absolute_error: 0.6780


In [193]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -1.132729 using {'units': 5}
-1.132729 (0.370539) with: {'units': 5}
-1.202227 (0.410874) with: {'units': 10}


In [ ]:
model = Sequential()
model.add(LSTM(units=10, input_shape=(1,features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')
history=model.fit(X,y, verbose=1, epochs=50, validation_split=0.25)

In [ ]:
# show_history(history)
# history.history
model.predict(X[:1]), y[1]

In [ ]:
grid_result.cv_results_

In [181]:
grid_result

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7ff009c1e8d0>,
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'units': [5, 10]}, pre_dispatch='2*n_jobs', refit=True,
       return_train_score='warn', scoring=None, verbose=0)